# 1. start with small idea of RAG - search by keywords matching

In [ ]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI
import functools
from concurrent.futures import ThreadPoolExecutor
import time

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

# Configuration

MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [3]:
# Performance improvement: Use caching for context loading
@functools.lru_cache(maxsize=None)
def load_context():
    """Load and cache context data to avoid repeated file I/O"""
    context = {}
    
    # Load employees with threading for faster I/O
    def load_employee_files():
        employee_context = {}
        employees = glob.glob("knowledge-base/employees/*")
        
        def load_single_employee(employee):
            name = employee.split(' ')[-1][:-3]
            try:
                with open(employee, "r", encoding="utf-8") as f:
                    return name, f.read()
            except Exception as e:
                print(f"Error loading {employee}: {e}")
                return name, ""
        
        # Use ThreadPoolExecutor for concurrent file loading
        with ThreadPoolExecutor(max_workers=4) as executor:
            results = executor.map(load_single_employee, employees)
            employee_context.update(dict(results))
        
        return employee_context
    
    # Load schools with threading
    def load_school_files():
        school_context = {}
        schools = glob.glob("knowledge-base/schools/*")
        
        def load_single_school(school):
            name = school.split(os.sep)[-1][:-3]
            try:
                with open(school, "r", encoding="utf-8") as f:
                    return name, f.read()
            except Exception as e:
                print(f"Error loading {school}: {e}")
                return name, ""
        
        with ThreadPoolExecutor(max_workers=4) as executor:
            results = executor.map(load_single_school, schools)
            school_context.update(dict(results))
        
        return school_context
    
    # Load both types concurrently
    with ThreadPoolExecutor(max_workers=2) as executor:
        employee_future = executor.submit(load_employee_files)
        school_future = executor.submit(load_school_files)
        
        context.update(employee_future.result())
        context.update(school_future.result())
    
    return context

In [ ]:
# Load context once at startup
context = load_context()
print(f"Loaded context with {len(context)} documents: {list(context.keys())}")

In [ ]:
context["Lan"]

In [5]:
system_message = (
    "Bạn là một chuyên gia tư vấn du học Hàn Quốc tại trung tâm Korea Study. "
    "Nhiệm vụ của bạn là trả lời các câu hỏi liên quan đến trung tâm, nhân viên, trường học, và thông tin visa một cách ngắn gọn và chính xác. "
    "Nếu bạn không biết câu trả lời, hãy nói rõ rằng bạn không biết. "
    "Tuyệt đối không bịa ra thông tin nếu không có ngữ cảnh liên quan được cung cấp."
)

In [6]:
def get_relevant_context(message):
    relevant_context = []
    for context_title, context_details in context.items():
        if context_title.lower() in message.lower():
            relevant_context.append(context_details)
    return relevant_context  

In [ ]:
get_relevant_context("Who is Timi?")

In [ ]:
get_relevant_context("who is Nguyễn Thị Lan")

In [ ]:
get_relevant_context("who is Nguyễn Thị Lan and introduce about Yonsei University")

In [ ]:
get_relevant_context("who is nguyen thi la and introduce about yonsei")

In [8]:
def add_context(message):
    """Add relevant context to message"""
    relevant_context = get_relevant_context(message)
    if relevant_context:
        message += "\n\nNhững thông tin sau có thể hữu ích cho việc trả lời câu hỏi này:\n\n"
        for relevant in relevant_context:
            message += relevant + "\n\n"
    return message

In [9]:
def chat(message, history):
    """Optimized chat function with better error handling"""
    try:
        messages = [{"role": "system", "content": system_message}] + history
        message = add_context(message)
        messages.append({"role": "user", "content": message})

        stream = openai.chat.completions.create(
            model=MODEL, 
            messages=messages, 
            stream=True,
            max_tokens=1000,  # Limit response length for faster generation
            temperature=0.7
        )

        response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content:
                response += chunk.choices[0].delta.content
                yield response
    except Exception as e:
        yield f"Xin lỗi, đã có lỗi xảy ra: {str(e)}"

In [ ]:
# Launch first version
print("Launching keyword-based RAG chatbot...")
view = gr.ChatInterface(chat, type="messages").launch()

# 2. RAG bigger idea with vector search - optimized version

In [3]:
# imports for langchain

from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

folders = glob.glob("knowledge-base/*")

#text_loader_kwargs = {'encoding': 'utf-8'}
# Nếu dòng trên không hoạt động, người dùng Windows có thể dùng dòng dưới thay thế
text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

print("Total documents loaded:", len(documents))


In [ ]:
documents[0]

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  # Slightly smaller chunks for better retrieval
    chunk_overlap=100,  # Reduced overlap for performance
    separators=["\n\n", "\n", ". ", " ", ""]  # Better separation
)

chunks = text_splitter.split_documents(documents)
print(f"Created {len(chunks)} chunks")

In [ ]:
chunks[9]

In [ ]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Các loại tài liệu đã tìm thấy: {', '.join(doc_types)}")

In [ ]:
for chunk in chunks:
    if 'Nguyễn Thị Lan' in chunk.page_content:
        print(chunk)
        print("_________")

In [14]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [15]:
# Đưa các đoạn văn bản (chunks) vào Vector Store, liên kết mỗi đoạn với một vector embedding

embeddings = OpenAIEmbeddings()

# Nếu bạn muốn sử dụng embeddings miễn phí từ HuggingFace (thay vì OpenAI),
# hãy thay dòng embeddings = OpenAIEmbeddings()
# bằng:
# from langchain.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [16]:
MODEL = "gpt-4o-mini"

# Đặt tên cho database vector (có thể tùy chọn)
db_name = "vector_db"

# Kiểm tra nếu database Chroma đã tồn tại, thì xóa collection để khởi động lại từ đầu
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

In [ ]:
# Tạo vector store bằng Chroma
vectorstore = Chroma.from_documents(
    documents=chunks,              # Danh sách các đoạn văn bản đã chia nhỏ
    embedding=embeddings,          # Hàm embedding (ví dụ: OpenAI hoặc HuggingFace)
    persist_directory=db_name      # Thư mục lưu trữ database
)
# Kiểm tra số lượng document đã được lưu vào vector store
print(f"Vectorstore created with {vectorstore._collection.count()} documents")


In [ ]:
# Lấy ra bộ sưu tập vector từ vectorstore
collection = vectorstore._collection

# Lấy 1 embedding từ database
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]

# Kiểm tra số chiều (số phần tử trong vector)
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

In [ ]:
sample_embedding

In [19]:
# Lấy toàn bộ vector, tài liệu và metadata từ collection
result = collection.get(include=['embeddings', 'documents', 'metadatas'])

# Đưa embedding vào mảng numpy
vectors = np.array(result['embeddings'])

# Lưu lại văn bản
documents = result['documents']

# Trích loại tài liệu từ metadata (giả sử có 'doc_type')
doc_types = [metadata['doc_type'] for metadata in result['metadatas']]

# Gán màu sắc tùy theo loại tài liệu
colors = [['blue', 'green', 'red', 'orange'][['company', 'employees', 'visas', 'schools'].index(t)] for t in doc_types]


In [30]:
# Con người chúng ta dễ hình dung mọi thứ trong không gian 2D hơn!
# Giảm số chiều của vector xuống 2D bằng t-SNE
# (T-distributed Stochastic Neighbor Embedding)

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Tạo biểu đồ scatter 2D
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Loại: {t}<br>Văn bản: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='Biểu đồ 2D Chroma Vector Store',
    scene=dict(xaxis_title='x', yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show(renderer="browser")

In [20]:
from langchain.memory import ConversationBufferWindowMemory  
from langchain.chains import ConversationalRetrievalChain

In [ ]:
# Tạo mô hình Chat với OpenAI
llm = ChatOpenAI(
    temperature=0.7, 
    model_name=MODEL,
)

# Thiết lập bộ nhớ hội thoại
memory = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True)

# Tạo retriever từ vector store (Chroma)
retriever = vectorstore.as_retriever()

# Kết nối tất cả thành một chuỗi hội thoại có khả năng truy xuất (RAG pipeline)
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory
)


In [21]:
# Test query with performance monitoring
def test_query_performance():
    """Test query with timing"""
    query = "Bạn có thể mô tả ngắn gọn về Korea study center không?"
    start_time = time.time()
    result = conversation_chain.invoke({"question": query})
    end_time = time.time()
    
    print(f"Query processed in {end_time - start_time:.2f} seconds")
    print("Answer:", result["answer"])
    if "source_documents" in result:
        print(f"Used {len(result['source_documents'])} source documents")

In [ ]:
test_query_performance()

In [22]:
# set up a new conversation memory for the chat
memory = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True)

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [23]:
# Wrapping in a function - note that history isn't used, as the memory is in the conversation_chain

def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [ ]:
# And in Gradio:

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

In [25]:
# Hãy cùng tìm hiểu xem điều gì được gửi phía sau hậu trường

from langchain_core.callbacks import StdOutCallbackHandler

llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

memory = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True)

retriever = vectorstore.as_retriever()

conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    retriever=retriever, 
    memory=memory, 
    callbacks=[StdOutCallbackHandler()]
)




In [26]:
# create a new Chat with OpenAI
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG; k is how many chunks to use
retriever = vectorstore.as_retriever(search_kwargs={"k": 30})

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    retriever=retriever, 
    memory=memory, 
    callbacks=[StdOutCallbackHandler()]
)


In [27]:
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]


In [ ]:

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

# 3 Improve RAG Ensemble Hybrid Retrieval.

In [ ]:
from langchain.schema import BaseRetriever, Document
from langchain.retrievers import EnsembleRetriever
from typing import List, Dict

# 1. Fix keyword retriever to search content, not just title
class KeywordRetriever(BaseRetriever):
    context_dict: Dict[str, str]

    def get_relevant_documents(self, query: str) -> List[Document]:
        relevant_docs = []
        for title, content in self.context_dict.items():
            if any(kw in content.lower() for kw in query.lower().split()):
                relevant_docs.append(Document(page_content=content, metadata={"source": title}))
        return relevant_docs

    async def aget_relevant_documents(self, query: str) -> List[Document]:
        return self.get_relevant_documents(query)

# 2. Use keyword + properly configured vector retriever
keyword_retriever = KeywordRetriever(context_dict=context)
vector_retriever = vectorstore.as_retriever(
    search_kwargs={"k": 30}
)

# 3. Ensemble
hybrid_retriever = EnsembleRetriever(
    retrievers=[keyword_retriever, vector_retriever],
    weights=[0.5, 0.5]
)

# 4. Conversation chain
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=hybrid_retriever,
    memory=memory,
    callbacks=[StdOutCallbackHandler()]
)


In [ ]:
query = "Nhân viên nào trong công ty đã tốt nghiệp Đại học Ngoại thương?"
result = conversation_chain.invoke({"question": query})
answer = result["answer"]
print("\nAnswer:", answer)

In [ ]:
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)